# Dataset Preparation Notebook
This notebook loads 3D-FRONT scenes, generates accurate semantic point clouds and color-coded 2D top-down layout maps.

### **1. Imports and Setup**

This first cell imports all the necessary libraries for the script. It includes standard libraries like `os` and `json`, data manipulation libraries like `numpy`, 3D processing with `trimesh`, and image handling with `PIL` (Pillow). Notably, `matplotlib` is configured with a non-interactive 'Agg' backend, which is essential for running the script in environments without a graphical user interface, such as a remote server or a Docker container.

In [1]:
import os
import json
import itertools
import shutil
import random
from pathlib import Path
from tqdm.notebook import tqdm  # Use notebook-friendly tqdm
import numpy as np

# Import matplotlib and set backend for headless operation
import matplotlib
matplotlib.use('Agg') # Use a non-interactive backend
import matplotlib.pyplot as plt

from typing import Union, List, Dict, Any, Tuple
import trimesh
from scipy.spatial.transform import Rotation
from PIL import Image, ImageDraw

---

### **2. Configuration**

Instead of using command-line arguments, you can set all the necessary paths and options here. The script can auto-detect the model directory and info file, but you can also specify them manually for more control.

In [2]:
# --- Main Configuration ---
# Set the output directory for generated pairs.
OUTPUT_PATH = Path("output_pairs")

# Run in headless mode (suppresses verbose logs). Set to False for detailed output.
HEADLESS_MODE = False

# --- Path Configuration ---
# The JSON file listing valid scenes to process.
# This file should be generated by a prior validation script.
SCENES_FILE_PATH = Path("valid_scenes.json")

# --- Optional: Manually specify dataset paths ---
# If you leave these as None, the script will try to auto-detect them.
# Otherwise, provide the full path to the required directory/file.
MANUAL_MODEL_DIR = None   # e.g., Path("/path/to/your/3D-FUTURE-model")
MANUAL_MODEL_INFO = None  # e.g., Path("/path/to/your/model_info.json")

# --- Slicing Parameters ---
# The height (Z-axis) where the cross-section slice begins.
SLICE_Z_POSITION = 0.1
# The thickness of the cross-section slice for walls.
SLICE_THICKNESS = 1.5

---

### **3. Path Setup Function**

The `setup_paths` function is responsible for finding and validating the necessary dataset directories and files. It prioritizes any manually set paths from the configuration cell above. If no manual paths are provided, it attempts to automatically locate the `3D-FUTURE-model` directory and the `model_info.json` file by searching up from the current working directory. This makes the notebook more portable.

In [3]:
def setup_paths(model_dir_arg: Path = None, model_info_arg: Path = None) -> Tuple[Path, Path]:
    """
    Sets up and validates necessary dataset paths.
    Prioritizes user-provided arguments, otherwise auto-detects.
    """
    print("--- 1. SETTING UP PATHS ---")
    
    # --- Determine Model Directory ---
    model_dir_candidate = None
    if model_dir_arg:
        print(f"Using provided model directory path: {model_dir_arg}")
        assert model_dir_arg.is_dir(), f"Provided model directory not found: {model_dir_arg}"
        MODEL_DIR = model_dir_arg
        model_dir_candidate = MODEL_DIR
    else:
        print("Auto-detecting model directory...")
        current_path = Path.cwd()
        for _ in range(4): # Search up to 4 levels
            candidate = next(current_path.glob("**/3D-FUTURE-model"), None)
            if candidate:
                model_dir_candidate = candidate
                break
            current_path = current_path.parent
        
        if not model_dir_candidate:
            raise FileNotFoundError("Could not auto-detect '3D-FUTURE-model' directory.")

        if (model_dir_candidate / "3D-FUTURE-model").is_dir():
            MODEL_DIR = model_dir_candidate / "3D-FUTURE-model"
        else:
            MODEL_DIR = model_dir_candidate
        print(f"Found model directory at: {MODEL_DIR}")

    # --- Determine Model Info File ---
    if model_info_arg:
        print(f"Using provided model info file path: {model_info_arg}")
        assert model_info_arg.is_file(), f"Provided model info file not found: {model_info_arg}"
        MODEL_INFO_FILE = model_info_arg
    else:
        print("Auto-detecting model info file...")
        assert model_dir_candidate is not None, "Cannot search for model_info.json without a model directory."
        MODEL_INFO_FILE = next(model_dir_candidate.glob("**/model_info.json"), None)
        if not MODEL_INFO_FILE:
            raise FileNotFoundError(f"Could not auto-detect 'model_info.json' within or near {model_dir_candidate}")
        print(f"Found model info file at: {MODEL_INFO_FILE}")

    print("✅ Dataset paths have been validated successfully.")
    return MODEL_DIR, MODEL_INFO_FILE

---

### **4. Helper Functions**

These are small utility functions used throughout the script.
* `get_dominant_direction`: Calculates the primary spatial relationship (e.g., "is above", "is to the East of") between two points.
* `get_final_label`: Determines the most reliable category label for a piece of furniture, checking both the model info file and the scene's item data.

In [4]:
def get_dominant_direction(vector: np.ndarray) -> str:
    """Determines the main cardinal or vertical direction of a vector."""
    if np.linalg.norm(vector) < 0.1: return "is co-located with"
    dx, dy, dz = vector
    abs_x, abs_y, abs_z = abs(dx), abs(dy), abs(dz)
    if abs_y > abs_x and abs_y > abs_z:
        return "is above" if dy > 0 else "is below"
    elif abs_x > abs_z:
        return "is to the East of" if dx > 0 else "is to the West of"
    else:
        return "is to the South of" if dz > 0 else "is to the North of"

def get_final_label(item: Dict, model_info_map: Dict) -> Union[str, None]:
    """Determines an object's definitive label using the validation script's logic."""
    jid = item.get('jid')
    label = model_info_map.get(jid, {}).get('category')
    if not label or label in ['None', 'Unknown']:
        label = item.get('title')

    if label and isinstance(label, str) and label.strip():
        return label.strip()
    return None

---

### **5. Core Logic: Image Generation**

The `generate_sliced_layout_render` function is the heart of the image creation process. It performs several key steps:
1.  **Load Geometry**: It loads all meshes (furniture, walls, floors) from a scene's metadata.
2.  **Orient Scene**: It calculates a transformation matrix to align the scene to a consistent top-down view, primarily by analyzing the floor's normal vector.
3.  **Slice Walls**: It converts all meshes into a colored point cloud. Critically, it only keeps points from the **wall** meshes that fall within a specific vertical slice (defined by `SLICE_Z_POSITION` and `SLICE_THICKNESS`). All furniture and floor points are kept.
4.  **Render Image**: It projects the final, filtered point cloud onto a 2D canvas, creating a top-down layout image where walls appear as a cross-section.

In [5]:
def generate_sliced_layout_render(scene_metadata: Dict, model_dir: Path, model_info_map: Dict, color_map: Dict, slice_z_position: float, slice_thickness: float, headless: bool = False) -> Tuple[Union[Image.Image, None], Union[np.ndarray, None]]:
    """Creates a colored 2D render where walls are sliced, using a consistent color map."""
    if not headless: print("  ▶️ Starting Sliced Layout Generation...")
    
    # 1. Load and Orient all 3D Meshes
    all_meshes_to_process = []
    furniture_info_map = {f['uid']: f for f in scene_metadata.get('furniture', [])}
    mesh_uid_map = {m['uid']: m for m in scene_metadata.get('mesh', [])}
    rooms = scene_metadata.get("scene", {}).get("room", [])
    if not rooms: return None, np.eye(4)

    for room_data in rooms:
        for child in room_data.get("children", []):
            ref_uid = child.get("ref")
            if not ref_uid: continue

            try:
                mesh, mesh_type, label = (None, 'misc', None)
                if ref_uid in furniture_info_map:
                    item_info = furniture_info_map[ref_uid]
                    jid = item_info.get('jid')
                    if not jid: continue
                    label = get_final_label(item_info, model_info_map)
                    if not label: continue
                    model_path = model_dir / jid / "raw_model.obj"
                    if not model_path.exists(): continue
                    mesh, mesh_type = trimesh.load(model_path, force='mesh', process=False), 'furniture'
                elif ref_uid in mesh_uid_map:
                    mesh_data = mesh_uid_map[ref_uid]
                    if 'Ceiling' in mesh_data.get("type", ""): continue
                    vertices = np.array(mesh_data["xyz"], dtype=np.float32).reshape(-1, 3)
                    faces = np.array(mesh_data["faces"], dtype=np.int32).reshape(-1, 3)
                    mesh, mesh_type = trimesh.Trimesh(vertices=vertices, faces=faces, process=False), 'floor' if 'Floor' in mesh_data.get("type", "") else 'wall'
                    label = mesh_type

                if not mesh or mesh.is_empty: continue

                transform = np.eye(4)
                transform[:3, 3] = child.get("pos", [0, 0, 0])
                transform[:3, :3] = Rotation.from_quat(child.get("rot", [0, 0, 0, 1])).as_matrix()
                mesh.apply_transform(transform)
                mesh.apply_scale(child.get("scale", [1, 1, 1]))
                all_meshes_to_process.append({'mesh': mesh, 'type': mesh_type, 'label': label})
            except Exception as e:
                print(f"      ❗️ ERROR loading item {ref_uid}: {e}")
                continue
    
    if not all_meshes_to_process:
        print("      ⚠️ No valid geometry was loaded for the scene.")
        return None, np.eye(4)

    # Orient scene to top-down view
    rotation_matrix = np.eye(4)
    floor_meshes = [item['mesh'] for item in all_meshes_to_process if item['type'] == 'floor']
    if floor_meshes:
        combined_floor = trimesh.util.concatenate(floor_meshes)
        if not combined_floor.is_empty:
            floor_normal = combined_floor.face_normals[np.argmax(combined_floor.area_faces)]
            if floor_normal[2] < 0: floor_normal *= -1
            rotation_matrix = trimesh.geometry.align_vectors(floor_normal, [0, 0, 1])
            for item in all_meshes_to_process:
                item['mesh'].apply_transform(rotation_matrix)

    # 2. Generate Full 3D Point Cloud with Consistent Colors
    meshes_to_render = [item['mesh'] for item in all_meshes_to_process]
    mesh_props = [{'type': item['type'], 'label': item['label']} for item in all_meshes_to_process]
    default_color = (128, 128, 128)
    mesh_colors_list = [color_map.get(item['label'], default_color) for item in mesh_props]
    
    full_scene_mesh = trimesh.util.concatenate(meshes_to_render)
    face_counts_cumulative = np.cumsum([len(m.faces) for m in meshes_to_render])
    points, face_indices = trimesh.sample.sample_surface(full_scene_mesh, 1_500_000)
    source_mesh_indices = np.searchsorted(face_counts_cumulative, face_indices, side='right')
    point_colors = np.array([mesh_colors_list[i] for i in source_mesh_indices])
    point_types = np.array([mesh_props[i]['type'] for i in source_mesh_indices])

    # 3. Filter Wall Points to Keep Only the Slice Band
    is_wall_mask = (point_types == 'wall')
    wall_points_z = points[is_wall_mask, 2]
    in_band_mask = (wall_points_z >= slice_z_position) & (wall_points_z <= slice_z_position + slice_thickness)
    wall_points_in_band_full_mask = np.zeros(len(points), dtype=bool)
    wall_points_in_band_full_mask[is_wall_mask] = in_band_mask
    final_keep_mask = ~is_wall_mask | wall_points_in_band_full_mask
    final_points, final_colors = points[final_keep_mask], point_colors[final_keep_mask]

    # 4. Project and Render the Final Image
    sort_order = np.argsort(final_points[:, 2])
    sorted_points, sorted_colors = final_points[sort_order], final_colors[sort_order]
    xy_coords = sorted_points[:, :2]
    min_coords, max_coords = points[:, :2].min(axis=0), points[:, :2].max(axis=0)
    img_size_px, point_radius = 1024, 2
    scale = img_size_px / max(1e-6, (max_coords - min_coords).max())
    img_shape = (np.ceil((max_coords - min_coords) * scale).astype(int) + 4)[::-1]
    
    canvas = Image.new('RGB', (img_shape[1], img_shape[0]), 'white')
    draw = ImageDraw.Draw(canvas)
    pixel_coords = np.round((xy_coords - min_coords) * scale).astype(int) + 2
    pixel_coords[:, 1] = img_shape[0] - 1 - pixel_coords[:, 1]

    for i in range(len(pixel_coords)):
        px, py = pixel_coords[i]
        draw.rectangle([px - point_radius, py - point_radius, px + point_radius, py + point_radius], fill=tuple(sorted_colors[i]))

    if not headless: print("  ✅ Image Generation Complete.")
    return canvas, rotation_matrix

---

### **6. Core Logic: Scene Graph Tokenization**

This function, `generate_scene_graph_tokens`, complements the image generation by creating a structured, textual description of the scene. It identifies all furniture within each room and generates a "scene graph" composed of relationship triplets:
* **Intra-room relationships**: Describes how objects are positioned relative to each other within the same room (e.g., `(sofa, is to the West of, end_table)`).
* **Inter-room relationships**: Describes how rooms are positioned relative to each other based on their centroids (e.g., `(LivingRoom, is to the North of, Bedroom)`).
* **Containment**: Explicitly states which objects are in which room (e.g., `(sofa, is_in, LivingRoom)`).

The final output is a list of these tokens, enclosed by special `[SCENE_START]` and `[SCENE_END]` markers.

In [6]:
def generate_scene_graph_tokens(scene_data: Dict, model_info_map: Dict, rotation_matrix: np.ndarray, headless: bool = False) -> List[Any]:
    """Generates a tokenized scene graph with spatial relationships."""
    if not headless: print("  ▶️ Starting Token Generation...")
    
    furniture_details_map = {item['uid']: item for item in scene_data.get('furniture', [])}
    all_rooms = scene_data.get("scene", {}).get("room", [])
    valid_rooms = []
    
    # Filter for rooms that contain valid, labeled furniture
    for i, room in enumerate(all_rooms):
        furniture_in_room = []
        for child in room.get('children', []):
            ref_id = child.get('ref')
            if 'pos' in child and ref_id and ref_id in furniture_details_map:
                item_info = furniture_details_map[ref_id]
                final_label = get_final_label(item_info, model_info_map)
                if final_label:
                    furniture_in_room.append({'pos': np.array(child['pos']), 'label': final_label})
        if furniture_in_room:
            valid_rooms.append({"id": i, "type": room.get('type', 'Unknown'), "furniture": furniture_in_room})

    if not valid_rooms: return []
    if not headless: print(f"      - Found {len(valid_rooms)} rooms with furniture to tokenize.")

    tokenized_scene = {"inter_room_relationships": [], "rooms": []}
    valid_room_centroids = []
    
    # Process intra-room relationships
    for room_data in valid_rooms:
        current_room_tokens = {"room_id": room_data["id"], "room_type": room_data["type"], "furniture": [item['label'] for item in room_data["furniture"]], "intra_room_relationships": []}
        furniture_list = room_data["furniture"]
        positions = np.array([f['pos'] for f in furniture_list])
        if len(furniture_list) > 1:
            for item1, item2 in itertools.combinations(furniture_list, 2):
                relationship = get_dominant_direction(item2['pos'] - item1['pos'])
                token = (item2['label'], relationship, item1['label'])
                current_room_tokens["intra_room_relationships"].append(token)
        tokenized_scene["rooms"].append(current_room_tokens)
        centroid = np.mean(positions, axis=0)
        rotated_centroid = trimesh.transform_points(centroid.reshape(1, -1), rotation_matrix)[0]
        valid_room_centroids.append({'id': room_data["id"], 'type': room_data["type"], 'centroid': rotated_centroid})
    
    # Process inter-room relationships
    if len(valid_room_centroids) > 1:
        for room1, room2 in itertools.combinations(valid_room_centroids, 2):
            relationship = get_dominant_direction(room2['centroid'] - room1['centroid'])
            token = (f"Room{room2['id']}_{room2['type']}", relationship, f"Room{room1['id']}_{room1['type']}")
            tokenized_scene["inter_room_relationships"].append(token)

    # Assemble final token list
    advanced_tokens = ["[SCENE_START]"]
    for room_data in tokenized_scene.get("rooms", []):
        advanced_tokens.append("[ROOM_START]")
        room_name = f"Room{room_data['room_id']}_{room_data['room_type']}"
        for furniture_item in room_data.get("furniture", []):
            advanced_tokens.append((furniture_item, "is_in", room_name))
        for triplet in room_data.get("intra_room_relationships", []):
            advanced_tokens.append(triplet)
        advanced_tokens.append("[ROOM_END]")
    for triplet in tokenized_scene.get("inter_room_relationships", []):
        advanced_tokens.append(triplet)
    advanced_tokens.append("[SCENE_END]")
    
    if not headless: print(f"  ✅ Token Generation Complete ({len(advanced_tokens)} tokens).")
    return advanced_tokens

---

### **7. Main Execution**

This final cell orchestrates the entire process.
1.  **Setup**: It calls `setup_paths` to initialize and validate all file paths. It then loads the `model_info.json` metadata and the list of scenes to process.
2.  **Color Map Generation**: It pre-scans all scenes to find every unique furniture label. It then creates a consistent, deterministic color map for all labels, ensuring that, for example, a "sofa" is the same color in every generated image. This color legend is saved to a JSON file.
3.  **Processing Loop**: It iterates through each validated scene. For each one, it calls `generate_sliced_layout_render` to create the image and `generate_scene_graph_tokens` to create the textual description. If both are successful, it saves the image (`.png`) and the tokens (`.json`) as a pair in the specified output directory.

A progress bar from `tqdm` provides feedback on the overall progress.

In [7]:
def run_processing():
    """Main function to generate pairs from a pre-validated list of scenes."""
    try:
        MODEL_DIR, MODEL_INFO_FILE = setup_paths(
            model_dir_arg=MANUAL_MODEL_DIR,
            model_info_arg=MANUAL_MODEL_INFO
        )
    except (AssertionError, FileNotFoundError) as e:
        print(f"❌ Error setting up paths: {e}")
        return

    # --- Load Metadata, Scene List, and Setup Output ---
    print("\n--- 2. LOADING METADATA AND SETTING UP OUTPUT ---")
    with open(MODEL_INFO_FILE, 'r', encoding='utf-8') as f:
        model_info_list = json.load(f)
    model_info_map = {item['model_id']: item for item in model_info_list if 'model_id' in item}
    print(f"Loaded metadata for {len(model_info_map)} models.")

    if not SCENES_FILE_PATH.exists():
        print(f"❌ Error: Scenes file not found at '{SCENES_FILE_PATH}'.")
        print("    Please run the validation script first or provide the correct path in the configuration cell.")
        return
    with open(SCENES_FILE_PATH, 'r', encoding='utf-8') as f:
        valid_scene_paths_str = json.load(f)
    valid_scenes = [Path(p) for p in valid_scene_paths_str]
    print(f"Loaded a list of {len(valid_scenes)} scenes to process from '{SCENES_FILE_PATH.name}'.")

    if OUTPUT_PATH.exists():
        print(f"⚠️ Warning: Output directory '{OUTPUT_PATH}' already exists. Its contents will be removed.")
        shutil.rmtree(OUTPUT_PATH)
    OUTPUT_PATH.mkdir(parents=True, exist_ok=True)
    print(f"✅ Output will be saved to: {OUTPUT_PATH}")

    # --- Pre-scan All Scenes to Build a Consistent Color Map ---
    print("\n--- 3. PRE-SCANNING SCENES FOR CONSISTENT COLOR SCHEME ---")
    all_possible_labels = set()
    for scene_path in tqdm(valid_scenes, desc="Scanning for categories"):
        with open(scene_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        for item in data.get('furniture', []):
            label = get_final_label(item, model_info_map)
            if label:
                all_possible_labels.add(label)

    sorted_labels = sorted(list(all_possible_labels))
    color_map = {}
    for label in sorted_labels:
        random.seed(hash(label)) # Use a deterministic seed
        color_map[label] = (random.randint(50, 200), random.randint(50, 200), random.randint(50, 200))
    
    # --- MODIFICATION START: Updated wall and floor colors ---
    color_map['wall'] = (0, 0, 0)          # Black
    color_map['floor'] = (255, 255, 255)   # White
    # --- MODIFICATION END ---

    print(f"Generated a consistent color map for {len(sorted_labels)} categories.")

    legend_path = OUTPUT_PATH / "color_legend.json"
    with open(legend_path, 'w', encoding='utf-8') as f:
        json.dump(color_map, f, indent=4)
    print(f"✅ Saved color legend to '{legend_path}'")

    # --- Process Scenes ---
    print("\n--- 4. PROCESSING SCENES AND GENERATING PAIRS ---")
    processed_count = 0
    failed_scenes = []

    for scene_path in tqdm(valid_scenes, desc="Generating pairs"):
        scene_id = scene_path.stem
        try:
            if not HEADLESS_MODE:
                print(f"\n------------------\n🎬 Processing Scene: {scene_id}\n------------------")

            with open(scene_path, 'r', encoding='utf-8') as f:
                scene_data = json.load(f)

            image, rotation_matrix = generate_sliced_layout_render(
                scene_metadata=scene_data, model_dir=MODEL_DIR,
                model_info_map=model_info_map, color_map=color_map,
                slice_z_position=SLICE_Z_POSITION, slice_thickness=SLICE_THICKNESS,
                headless=HEADLESS_MODE
            )
            if image is None:
                tqdm.write(f"❌ Skipping scene {scene_id}: Could not generate image.")
                continue

            tokens = generate_scene_graph_tokens(
                scene_data, model_info_map, rotation_matrix, headless=HEADLESS_MODE
            )
            if not tokens:
                tqdm.write(f"❌ Skipping scene {scene_id}: Could not generate tokens.")
                continue

            image_path = OUTPUT_PATH / f"{scene_id}_image.png"
            image.save(image_path)
            
            tokens_path = OUTPUT_PATH / f"{scene_id}_tokens.json"
            with open(tokens_path, 'w', encoding='utf-8') as f:
                json.dump(tokens, f, indent=2)

            if not HEADLESS_MODE:
                print(f"  💾 Saved pair: {image_path.name}, {tokens_path.name}")
            
            processed_count += 1
        
        except Exception as e:
            tqdm.write(f"\n❌❌❌ UNEXPECTED ERROR processing scene {scene_id}. Skipping. ❌❌❌")
            tqdm.write(f"      Error details: {e}")
            failed_scenes.append(scene_id)

    print(f"\n\n=====================================================")
    print(f"🎉 All tasks complete. {processed_count} image-token pairs saved in '{OUTPUT_PATH}'.")
    if failed_scenes:
        print(f"\n⚠️ Encountered critical errors in {len(failed_scenes)} scene(s):")
        for f_id in failed_scenes:
            print(f"    - {f_id}")
    print(f"=====================================================")


# Execute the main processing function
run_processing()

# Execute the main processing function
run_processing()

--- 1. SETTING UP PATHS ---
Auto-detecting model directory...
Found model directory at: c:\Users\Hagai.LAPTOP-QAG9263N\Desktop\Thesis\datasets\3D-FRONT_FUTURE\3D-FUTURE-model\3D-FUTURE-model
Auto-detecting model info file...
Found model info file at: c:\Users\Hagai.LAPTOP-QAG9263N\Desktop\Thesis\datasets\3D-FRONT_FUTURE\3D-FUTURE-model\model_info.json
✅ Dataset paths have been validated successfully.

--- 2. LOADING METADATA AND SETTING UP OUTPUT ---
Loaded metadata for 16563 models.
Loaded a list of 4843 scenes to process from 'valid_scenes.json'.
✅ Output will be saved to: output_pairs

--- 3. PRE-SCANNING SCENES FOR CONSISTENT COLOR SCHEME ---


Scanning for categories:   0%|          | 0/4843 [00:00<?, ?it/s]

Generated a consistent color map for 282 categories.
✅ Saved color legend to 'output_pairs\color_legend.json'

--- 4. PROCESSING SCENES AND GENERATING PAIRS ---


Generating pairs:   0%|          | 0/4843 [00:00<?, ?it/s]


------------------
🎬 Processing Scene: 00004f89-9aa5-43c2-ae3c-129586be8aaa
------------------
  ▶️ Starting Sliced Layout Generation...
  ✅ Image Generation Complete.
  ▶️ Starting Token Generation...
      - Found 7 rooms with furniture to tokenize.
  ✅ Token Generation Complete (211 tokens).
  💾 Saved pair: 00004f89-9aa5-43c2-ae3c-129586be8aaa_image.png, 00004f89-9aa5-43c2-ae3c-129586be8aaa_tokens.json

------------------
🎬 Processing Scene: 0003d406-5f27-4bbf-94cd-1cff7c310ba1
------------------
  ▶️ Starting Sliced Layout Generation...
  ✅ Image Generation Complete.
  ▶️ Starting Token Generation...
      - Found 8 rooms with furniture to tokenize.
  ✅ Token Generation Complete (627 tokens).
  💾 Saved pair: 0003d406-5f27-4bbf-94cd-1cff7c310ba1_image.png, 0003d406-5f27-4bbf-94cd-1cff7c310ba1_tokens.json

------------------
🎬 Processing Scene: 00154c06-2ee2-408a-9664-b8fd74742897
------------------
  ▶️ Starting Sliced Layout Generation...
  ✅ Image Generation Complete.
  ▶️ Starti

C:\Users\Hagai.LAPTOP-QAG9263N\AppData\Local\Temp\ipykernel_23276\1891665349.py:93: RuntimeWarning: invalid value encountered in cast
  img_shape = (np.ceil((max_coords - min_coords) * scale).astype(int) + 4)[::-1]



❌❌❌ UNEXPECTED ERROR processing scene 00fc9d81-7397-4578-a865-920c4d89b44b. Skipping. ❌❌❌
      Error details: Width and height must be >= 0

------------------
🎬 Processing Scene: 00fe8139-76e8-42b4-bb41-7f9f085ac351
------------------
  ▶️ Starting Sliced Layout Generation...
  ✅ Image Generation Complete.
  ▶️ Starting Token Generation...
      - Found 10 rooms with furniture to tokenize.
  ✅ Token Generation Complete (227 tokens).
  💾 Saved pair: 00fe8139-76e8-42b4-bb41-7f9f085ac351_image.png, 00fe8139-76e8-42b4-bb41-7f9f085ac351_tokens.json

------------------
🎬 Processing Scene: 0106f9d2-5779-457b-9b8b-72942373d42e
------------------
  ▶️ Starting Sliced Layout Generation...
  ✅ Image Generation Complete.
  ▶️ Starting Token Generation...
      - Found 5 rooms with furniture to tokenize.
  ✅ Token Generation Complete (205 tokens).
  💾 Saved pair: 0106f9d2-5779-457b-9b8b-72942373d42e_image.png, 0106f9d2-5779-457b-9b8b-72942373d42e_tokens.json

------------------
🎬 Processing Scen

KeyboardInterrupt: 